In [1]:
import numpy as np

G = np.array([
    [0, 1/2, 0, 0, 1/2, 0, 0, 0, 0, 0, 0, 0],       # A
    [1/3, 0, 1/3, 1/3, 0, 0, 0, 0, 0, 0, 0, 0],     # B
    [0, 1/2, 0, 1/2, 0, 0, 0, 0, 0, 0, 0, 0],       # C
    [1/3, 0, 0, 0, 0, 1/3, 0, 0, 1/3, 0, 0, 0],     # D
    [0, 0, 0, 1/2, 0, 0, 0, 1/2, 0, 0, 0, 0],       # E
    [0, 0, 0, 0, 1/2, 0, 0, 0, 1/2, 0, 0, 0],       # F
    [0, 0, 1/4, 1/4, 0, 0, 0, 0, 1/4, 0, 0, 1/4],   # G
    [0, 0, 0, 0, 1/5, 1/5, 0, 0, 1/5, 1/5, 1/5, 0], # H
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],           # I
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],           # J
    [0, 0, 0, 0, 0, 0, 0, 0, 1/3, 1/3, 0, 1/3],     # K
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]            # L
])

#E: Transpose
G_T = G.T

#F: Eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(G_T)

#G: Dominant eigenvector (corresponding to eigenvalue 1)
dominant_eigenvector = eigenvectors[:, np.isclose(eigenvalues, 1)].flatten()

dominant_eigenvector = np.real(dominant_eigenvector)

#H: Normalize dominant eigenvector
dominant_eigenvector /= np.sum(dominant_eigenvector)

# All entries are positive?
if np.any(dominant_eigenvector < 0):
    dominant_eigenvector = -dominant_eigenvector

print("Dominant eigenvector (PageRank vector):", dominant_eigenvector)
print("Eigenvalues:", eigenvalues)

#I: Ranking pages
ranking = np.argsort(dominant_eigenvector)[::-1]  
pages = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']
ranked_pages = [pages[i] for i in ranking]

print("Page Ranking:", ranked_pages)

Dominant eigenvector (PageRank vector): []
Eigenvalues: [ 0.        +0.j          0.99057675+0.j          0.54352322+0.20239006j
  0.54352322-0.20239006j -0.11200556+0.53180378j -0.11200556-0.53180378j
 -0.47778499+0.45864795j -0.47778499-0.45864795j -0.53254921+0.j
  0.08742385+0.j         -0.22645837+0.07939335j -0.22645837-0.07939335j]
Page Ranking: []


In [2]:
#1)

import numpy as np

G = np.array([
    [0, 1/2, 0, 0, 1/2, 0, 0, 0, 0, 0, 0, 0],       # A
    [1/3, 0, 1/3, 1/3, 0, 0, 0, 0, 0, 0, 0, 0],     # B
    [0, 1/2, 0, 1/2, 0, 0, 0, 0, 0, 0, 0, 0],       # C
    [1/3, 0, 0, 0, 0, 1/3, 0, 0, 1/3, 0, 0, 0],     # D
    [0, 0, 0, 1/2, 0, 0, 0, 1/2, 0, 0, 0, 0],       # E
    [0, 0, 0, 0, 1/2, 0, 0, 0, 1/2, 0, 0, 0],       # F
    [0, 0, 1/4, 1/4, 0, 0, 0, 0, 1/4, 0, 0, 1/4],   # G
    [0, 0, 0, 0, 1/5, 1/5, 0, 0, 1/5, 1/5, 1/5, 0], # H
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],           # I
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],           # J
    [0, 0, 0, 0, 0, 0, 0, 0, 1/3, 1/3, 0, 1/3],     # K
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]            # L
])

# Check row sums
row_sums = G.sum(axis=1)
row_sums

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.])

In [3]:
#2)

# Eigenvalue decomposition
eigvals, eigvecs = np.linalg.eig(G.T)

# Find the eigenvector corresponding to the eigenvalue 1
eig_index = np.argmin(np.abs(eigvals - 1))
pi = eigvecs[:, eig_index].real

# Normalize the eigenvector
pi = pi / np.sum(pi)
pi

array([0.06974077, 0.06964627, 0.06823918, 0.13760448, 0.06918413,
       0.05335517, 0.17752289, 0.03492114, 0.12746202, 0.00942325,
       0.00705067, 0.17585004])

In [4]:
#3)

# Fix dangling nodes
dangling_nodes = np.where(row_sums == 0)[0]
for node in dangling_nodes:
    G[node, :] = 1 / G.shape[1]

# Recompute row sums to ensure they sum to 1
G = G / G.sum(axis=1, keepdims=True)

# Eigenvalue decomposition again
eigvals, eigvecs = np.linalg.eig(G.T)

# Find the eigenvector corresponding to the eigenvalue 1
eig_index = np.argmin(np.abs(eigvals - 1))
pi = eigvecs[:, eig_index].real

# Normalize the eigenvector
pi = pi / np.sum(pi)
pi

array([0.0698299 , 0.0698299 , 0.06803939, 0.13697404, 0.0698299 ,
       0.05371531, 0.17547001, 0.03581021, 0.12712623, 0.01074306,
       0.0080573 , 0.17457475])

In [6]:
#4)

alpha = 0.85
n = G.shape[1]
E = np.ones((n, n)) / n
G_tilde = alpha * G + (1 - alpha) * E

# Eigenvalue decomposition of G_tilde
eigvals, eigvecs = np.linalg.eig(G_tilde.T)

# Find the eigenvector corresponding to the eigenvalue 1
eig_index = np.argmin(np.abs(eigvals - 1))
pi_tilde = eigvecs[:, eig_index].real

# Normalize the eigenvector
pi_tilde = pi_tilde / np.sum(pi_tilde)
pi_tilde

array([0.0715815 , 0.07325075, 0.06650942, 0.12799417, 0.07816056,
       0.05894978, 0.14679029, 0.04778035, 0.12162373, 0.02911212,
       0.02268477, 0.15556256])

In [8]:
#5)

# Term-document matrix (T) and query vector (q)
keywords = [
    ["Apples", "Bananas", "Broccoli", "Cabbage", "Kumquats", "Strawberries"],  # Page A
    ["Oranges", "Plums", "Coconuts", "Kumquats", "Blueberries", "Cherries", "Strawberries"],  # Page B
    ["Lettuce", "Spinach", "Bananas", "Blackberries", "Peas", "Strawberries"],  # Page C
    ["Oranges", "Onions", "Celery", "Kumquats", "Corn", "Radishes"],  # Page D
    ["Pineapples", "Plums", "Corn", "Cherries", "Broccoli", "Peas", "Strawberries"],              # Page E
    ["Lettuce", "Onions", "Coconuts", "Spinach", "Peas", "Strawberries"],              # Page F
    ["Apples", "Onions", "Broccoli", "Corn", "Cabbage", "Peas"],  # Page G
    ["Plums", "Blueberries", "Raspberries", "Blackberries", "Strawberries"],              # Page H
    ["Apples", "Cucumbers", "Carrots", "Spinach", "Corn", "Black Beans", "Cabbage"],              # Page I
    ["Mushrooms", "Carrots", "Lettuce", "Radishes", "Peppers", "Broccoli", "Spinach"],             # Page J
    ["Carrots", "Lettuce", "Celery", "Onions", "Cabbage", "Peas"], # Page K
    ["Broccoli", "Cabbage", "Carrots", "Spinach", "Corn", "Peas"]              # Page L
]

# Creating the term-document matrix T
all_keywords = list(set(sum(keywords, [])))
T = np.zeros((len(all_keywords), len(keywords)))

for j, page_keywords in enumerate(keywords):
    for keyword in page_keywords:
        i = all_keywords.index(keyword)
        T[i, j] = 1

# Example user query
query_keywords = ["Apples", "Oranges"]
q = np.zeros(len(all_keywords))
for keyword in query_keywords:
    if keyword in all_keywords:
        i = all_keywords.index(keyword)
        q[i] = 1

# Compute d^T = q^T T
d = np.dot(q, T)

# Responsive webpages
responsive_webpages = np.where(d > 0)[0]

# Rank the responsive webpages using pi_tilde
pi_tilde = pi_tilde
responsive_ranking = np.argsort(-pi_tilde[responsive_webpages])
responsive_webpages_ranked = responsive_webpages[responsive_ranking]

responsive_webpages_ranked

array([6, 3, 8, 1, 0], dtype=int64)